In [3]:
import torch
from torch import nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os

data_dir = "./data"
print('Folders :', os.listdir(data_dir))
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}



animal_dataset = ImageFolder(data_dir+ "/training",
                             transform=transforms.Compose([
                                        transforms.Grayscale(num_output_channels=1),
                                        transforms.ToTensor(), 
                                        transforms.Resize((200, 200)), 
                                        # transforms.Lambda(lambda x: x.permute(1, 2, 0)),
                                        ])
                             )
print('Size of training dataset :', len(animal_dataset))

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def show_example(img, label):
    print('Label: ', animal_dataset.classes[label], "("+str(label)+")")
    plt.imshow(img, cmap='gray')

image = animal_dataset[500]
print(image[0].shape)
# show_example(*image)

# scaled_image = nn.functional.interpolate(image[0], size=200, mode="linear", align_corners=True)
# show_example(scaled_image, 1)


torch.Size([1, 200, 200])


In [ ]:
batch_size = 100
keep_prob = 1
training_data = DataLoader(animal_dataset, batch_size=batch_size)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
sample = animal_dataset[2000]
print(sample[0].shape)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, padding=2, stride=1),          # Conv -> (?, 200, 200, 32)
            nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),                                            # Pool -> (?, 100, 100, 32)
            torch.nn.Dropout(p=1 - keep_prob))
            
                                                                                                    # L2 ImgIn shape=(?, 100, 100, 32)    
        self.layer2 = nn.Sequential(                                                             
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2, stride=1),         # Conv -> (?, 100, 100, 64)
            nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),                                            # Pool -> (?, 50, 50, 64)
            torch.nn.Dropout(p=1 - keep_prob))

                                                                                                    # L3 ImgIn shape=(?, 50, 50, 64)    
        self.layer3 = nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=5, padding=2, stride=1),                           # Conv -> (?, 50, 50, 128)
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),                                            # Pool -> (?, 25, 25, 128)
            torch.nn.Dropout(p=1 - keep_prob))

                                                                                                    # L4 ImgIn shape=(?, 25, 25, 128)    
        self.layer4 = nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=5, padding=2, stride=1),                           # Conv -> (?, 25, 25, 256)
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=3, stride=3, padding=1),                                  # Pool -> (?, 9, 9, 256)
            torch.nn.Dropout(p=1 - keep_prob))

        # L5 FC 9x9x256 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(9 * 9 * 256, 625, bias=True)         # mat = 20736x625
        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.layer5 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - keep_prob))
        # L6 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 3, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight) # initialize parameters


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        # out5 = self.layer5(out4)
        out = self.fc1(out)
        out = self.fc2(out)
        return out


#instantiate CNN model
model = ConvNet()
model

Using cpu device
torch.Size([1, 200, 200])


/tmp/ipykernel_842/2809059464.py:37: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc1.weight)


ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (layer4): Sequential(
    (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
    (3): Dropout(p=0, inplace=False)
  )
  (fc1): Linear(in_features=2

In [ ]:
learning_rate = 0.001
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed.
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
list(model.parameters())[0].shape 

torch.Size([32, 1, 5, 5])

In [6]:
from torch.autograd import Variable
print('Training the Deep Learning network ...')
train_cost = []
train_accu = []

training_epochs = 15
total_batch = len(animal_dataset) // batch_size

# print('Size of the training dataset is {}'.format(animal_dataset.data.size()))
# print('Size of the testing dataset'.format(animal_dataset.data.size()))
print('Batch size is : {}'.format(batch_size))
print('Total number of batches is : {0:2.0f}'.format(total_batch))
print('\nTotal number of epochs is : {0:2.0f}'.format(training_epochs))

for epoch in range(training_epochs):
    avg_cost = 0
    for i, (batch_X, batch_Y) in enumerate(training_data):
        X = Variable(batch_X)    # image is already size of (28x28), no reshape
        Y = Variable(batch_Y)    # label is not one-hot encoded

        optimizer.zero_grad() # <= initialization of the gradients
        
        # forward propagation
        hypothesis = model(X)
        cost = criterion(hypothesis, Y) # <= compute the loss function
        
        # Backward propagation
        cost.backward() # <= compute the gradient of the loss/cost function     
        optimizer.step() # <= Update the gradients
             
        # Print some performance to monitor the training
        prediction = hypothesis.data.max(dim=1)[1]
        train_accu.append(((prediction.data == Y.data).float().mean()).item())
        train_cost.append(cost.item())   
        if i % 200 == 0:
            print("Epoch= {},\t batch = {},\t cost = {:2.4f},\t accuracy = {}".format(epoch+1, i, train_cost[-1], train_accu[-1]))
       
        avg_cost += cost.data / total_batch

    print("[Epoch: {:>4}], averaged cost = {:>.9}".format(epoch + 1, avg_cost.item()))


print('Learning Finished!')

Training the Deep Learning network ...
Batch size is : 100
Total number of batches is : 80

Total number of epochs is : 15
Epoch= 1,	 batch = 0,	 cost = 2.2543,	 accuracy = 0.12999999523162842


KeyboardInterrupt: 